In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
import scipy.stats

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras,scipy:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
matplotlib 3.2.0
numpy 1.16.2
pandas 1.0.1
sklearn 0.20.3
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf
scipy 1.4.1


In [3]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [4]:
# 分割数据集的工具
from sklearn.model_selection import train_test_split

# 分割训练集和测试集，比例为3:1
x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7) 
# 分割训练集和开发集，比例是3:1，总的比例是9:3:4
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)


(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [5]:
# 标准归一化工具
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [8]:
learning_rates = [1e-4,3*1e-4,1e-3,3*1e-3,1e-2,3*1e-2]

def bulid_model(hidden_layers,layer_size,learning_rate):
    model_keras = keras.models.Sequential();
    model_keras.add(keras.layers.Dense(layer_size, activation='relu',
                                       input_shape=x_train.shape[1:]))
    for _ in range(hidden_layers-1):
        model_keras.add(keras.layers.Dense(layer_size, activation='relu'))
    model_keras.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate)
    model_keras.compile(loss="mse", optimizer=optimizer)
    return model_keras

model = keras.wrappers.scikit_learn.KerasRegressor(bulid_model)

params = {'hidden_layers' : np.arange(1,5),
          'layer_size'   : np.arange(1,100),
          'learning_rate': scipy.stats.reciprocal(1e-4,1e-2)}

callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2),
             keras.callbacks.TensorBoard(log_dir='logs/regression-callbacks')]

random_search_cv = sklearn.model_selection.RandomizedSearchCV(model,
                                                              param_distributions=params,
                                                              cv=3)

history = random_search_cv.fit(x_train_scaled, y_train,
                validation_data = (x_valid_scaled, y_valid),
                epochs = 100,
                callbacks = callbacks)


Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 100us/sample - loss: 4.3369 - val_loss: 3.5388
Epoch 2/100
7740/7740 [==============================] - 0s 55us/sample - loss: 2.7588 - val_loss: 2.3128
Epoch 3/100
7740/7740 [==============================] - 0s 57us/sample - loss: 1.8741 - val_loss: 1.6719
Epoch 4/100
7740/7740 [==============================] - 0s 56us/sample - loss: 1.4357 - val_loss: 1.3510
Epoch 5/100
7740/7740 [==============================] - 1s 82us/sample - loss: 1.1960 - val_loss: 1.1534
Epoch 6/100
7740/7740 [==============================] - 1s 113us/sample - loss: 1.0361 - val_loss: 1.0124
Epoch 7/100
7740/7740 [==============================] - 1s 69us/sample - loss: 0.9217 - val_loss: 0.9095
Epoch 8/100
7740/7740 [==============================] - 0s 63us/sample - loss: 0.8375 - val_loss: 0.8349
Epoch 9/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.7763 - val_loss: 0.7

7740/7740 [==============================] - 0s 52us/sample - loss: 0.6962 - val_loss: 0.7513
Epoch 14/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6854 - val_loss: 0.7420
Epoch 15/100
7740/7740 [==============================] - 0s 51us/sample - loss: 0.6763 - val_loss: 0.7339
Epoch 16/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6685 - val_loss: 0.7267
Epoch 17/100
7740/7740 [==============================] - 0s 51us/sample - loss: 0.6614 - val_loss: 0.7202
Epoch 18/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6551 - val_loss: 0.7141
Epoch 19/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6492 - val_loss: 0.7084
Epoch 20/100
7740/7740 [==============================] - 0s 51us/sample - loss: 0.6437 - val_loss: 0.7029
Epoch 21/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6383 - val_loss: 0.6976
Epoch 22/100
7740/7740 [==========================

Epoch 13/100
7740/7740 [==============================] - 0s 51us/sample - loss: 0.7010 - val_loss: 0.7613
Epoch 14/100
7740/7740 [==============================] - 0s 51us/sample - loss: 0.6896 - val_loss: 0.7520
Epoch 15/100
7740/7740 [==============================] - 0s 50us/sample - loss: 0.6800 - val_loss: 0.7441
Epoch 16/100
7740/7740 [==============================] - 1s 89us/sample - loss: 0.6715 - val_loss: 0.7370
Epoch 17/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.6638 - val_loss: 0.7306
Epoch 18/100
7740/7740 [==============================] - 1s 101us/sample - loss: 0.6569 - val_loss: 0.7245
Epoch 19/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.6505 - val_loss: 0.7187
Epoch 20/100
7740/7740 [==============================] - 0s 65us/sample - loss: 0.6444 - val_loss: 0.7130
Epoch 21/100
7740/7740 [==============================] - 1s 92us/sample - loss: 0.6387 - val_loss: 0.7076
Epoch 22/100
7740/7740 [============

7740/7740 [==============================] - 0s 61us/sample - loss: 0.6162 - val_loss: 0.6316
Epoch 5/100
7740/7740 [==============================] - 1s 71us/sample - loss: 0.5834 - val_loss: 0.5982
Epoch 6/100
7740/7740 [==============================] - 1s 76us/sample - loss: 0.5565 - val_loss: 0.5787
Epoch 7/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.5345 - val_loss: 0.5449
Epoch 8/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5128 - val_loss: 0.5241
Epoch 9/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.4950 - val_loss: 0.5113
Epoch 10/100
7740/7740 [==============================] - 1s 109us/sample - loss: 0.4795 - val_loss: 0.4936
Epoch 11/100
7740/7740 [==============================] - 1s 71us/sample - loss: 0.4643 - val_loss: 0.4737
Epoch 12/100
7740/7740 [==============================] - 1s 97us/sample - loss: 0.4516 - val_loss: 0.4637
Epoch 13/100
7740/7740 [==============================

7740/7740 [==============================] - 0s 57us/sample - loss: 0.4439 - val_loss: 0.4801
Epoch 10/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.4311 - val_loss: 0.4664
Epoch 11/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.4207 - val_loss: 0.4558
Epoch 12/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.4105 - val_loss: 0.4443
Epoch 13/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.4027 - val_loss: 0.4352
Epoch 14/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.3948 - val_loss: 0.4298
Epoch 15/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.3878 - val_loss: 0.4216
Epoch 16/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.3825 - val_loss: 0.4139
Epoch 17/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.3774 - val_loss: 0.4090
Epoch 18/100
7740/7740 [==========================

...

Epoch 36/100
7740/7740 [==============================] - 0s 22us/sample - loss: 0.3540
Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 82us/sample - loss: 2.1128 - val_loss: 1.0631
Epoch 2/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.8113 - val_loss: 0.7481
Epoch 3/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.6531 - val_loss: 0.6862
Epoch 4/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.6120 - val_loss: 0.6534
Epoch 5/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5839 - val_loss: 0.6256
Epoch 6/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5593 - val_loss: 0.5995
Epoch 7/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5373 - val_loss: 0.5804
Epoch 8/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5178 - val_loss: 0.5588
Epoch 9/100
7740

Epoch 36/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6128 - val_loss: 0.6456
Epoch 37/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.6097 - val_loss: 0.6423
Epoch 38/100
7740/7740 [==============================] - 0s 54us/sample - loss: 0.6066 - val_loss: 0.6392
Epoch 39/100
7740/7740 [==============================] - 0s 54us/sample - loss: 0.6037 - val_loss: 0.6362
Epoch 40/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.6007 - val_loss: 0.6332
Epoch 41/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.5979 - val_loss: 0.6303
Epoch 42/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.5950 - val_loss: 0.6273
Epoch 43/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.5924 - val_loss: 0.6245
Epoch 44/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.5897 - val_loss: 0.6218
Epoch 45/100
7740/7740 [=============

Epoch 40/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.5579 - val_loss: 0.6059
Epoch 41/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.5548 - val_loss: 0.6027
Epoch 42/100
7740/7740 [==============================] - 0s 55us/sample - loss: 0.5519 - val_loss: 0.5995
Epoch 43/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.5490 - val_loss: 0.5965
Epoch 44/100
7740/7740 [==============================] - 0s 55us/sample - loss: 0.5462 - val_loss: 0.5935
Epoch 45/100
7740/7740 [==============================] - 0s 52us/sample - loss: 0.5435 - val_loss: 0.5906
Epoch 46/100
7740/7740 [==============================] - 0s 53us/sample - loss: 0.5409 - val_loss: 0.5877
Epoch 47/100
7740/7740 [==============================] - 1s 73us/sample - loss: 0.5383 - val_loss: 0.5850
Epoch 48/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.5358 - val_loss: 0.5822
Epoch 49/100
7740/7740 [=============

Epoch 48/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.5139 - val_loss: 0.5701
Epoch 49/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.5109 - val_loss: 0.5666
Epoch 50/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.5080 - val_loss: 0.5632
Epoch 51/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5051 - val_loss: 0.5598
Epoch 52/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.5023 - val_loss: 0.5565
Epoch 53/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.4996 - val_loss: 0.5533
Epoch 54/100
7740/7740 [==============================] - 0s 55us/sample - loss: 0.4969 - val_loss: 0.5502
Epoch 55/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.4943 - val_loss: 0.5471
Epoch 56/100
7740/7740 [==============================] - 1s 70us/sample - loss: 0.4917 - val_loss: 0.5441
Epoch 57/100
7740/7740 [=============

7740/7740 [==============================] - 0s 59us/sample - loss: 0.3973 - val_loss: 0.4194
Epoch 15/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.3928 - val_loss: 0.4145
Epoch 16/100
7740/7740 [==============================] - 1s 71us/sample - loss: 0.3892 - val_loss: 0.4111
Epoch 17/100
7740/7740 [==============================] - 1s 151us/sample - loss: 0.3848 - val_loss: 0.4073
Epoch 18/100
7740/7740 [==============================] - 0s 63us/sample - loss: 0.3807 - val_loss: 0.4038
Epoch 19/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.3775 - val_loss: 0.4107
Epoch 20/100
7740/7740 [==============================] - 1s 66us/sample - loss: 0.3743 - val_loss: 0.3949
Epoch 21/100
7740/7740 [==============================] - 0s 63us/sample - loss: 0.3712 - val_loss: 0.3933
Epoch 22/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.3686 - val_loss: 0.3905
Epoch 23/100
7740/7740 [=========================

In [9]:
help(history)

Help on RandomizedSearchCV in module sklearn.model_selection._search object:

class RandomizedSearchCV(BaseSearchCV)
 |  RandomizedSearchCV(estimator, param_distributions, n_iter=10, scoring=None, fit_params=None, n_jobs=None, iid='warn', refit=True, cv='warn', verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score='raise-deprecating', return_train_score='warn')
 |  
 |  Randomized search on hyper parameters.
 |  
 |  RandomizedSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated search over parameter settings.
 |  
 |  In contrast to GridSearchCV, not all parameter values are tried out, but
 |  rather a fixed number of parameter settings is sampled from the specified
 |  distributions. The number of parameter se

In [10]:
print(history.best_estimator_)
print(history.best_index_)
print(history.best_params_)
print(history.best_score_)

4
{'hidden_layers': 4, 'layer_size': 57, 'learning_rate': 0.002784457410995758}
-0.3414070444565


In [16]:
model = history.best_estimator_.model
model.evaluate(x=x_test_scaled,y=y_test)

5160/5160 [==============================] - 0s 29us/sample - loss: 0.3729


0.3729351328324902

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
plot_learning_curves(model)